In [ ]:
import zipfile

zip_ref = zipfile.ZipFile("all.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

zip_ref = zipfile.ZipFile("train.zip", 'r')
zip_ref.extractall(".")
zip_ref = zipfile.ZipFile("test.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

In [1]:
from keras.models import Model
from keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout, Lambda
from keras.applications import inception_resnet_v2
# from keras.preprocessing.image import *

import numpy as np
from tqdm import tqdm
import cv2

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
n = 25000
img_height, img_width = 299, 299
X = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
y = np.zeros((n, 1), dtype=np.uint8)

for i in tqdm(range(12500)): # n/2
    X[i] = cv2.resize(cv2.imread('train/cat.%d.jpg' % i), (img_height, img_width))
    X[i+12500] = cv2.resize(cv2.imread('train/dog.%d.jpg' % i), (img_height, img_width)) # n/2

y[12500:] = 1

100%|██████████| 12500/12500 [01:09<00:00, 180.28it/s]


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [4]:
i_input_tensor = Input((img_height, img_width, 3))
# x_input_tensor = inception_resnet_v2.preprocess_input(i_input_tensor)	# 错误的处理
x_input_tensor = Lambda(inception_resnet_v2.preprocess_input)(i_input_tensor)

base_model = inception_resnet_v2.InceptionResNetV2(input_tensor=x_input_tensor, weights='imagenet', include_top=False)

In [5]:
for layers in base_model.layers:
    layers.trainable = False

i_output = GlobalAveragePooling2D()(base_model.output)
i_output = Dropout(0.25)(i_output) # 0.25 比 0.5 好
i_predictions = Dense(1, activation='sigmoid')(i_output)
model = Model(base_model.input, i_predictions)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [6]:
model.fit(X_train, y_train, batch_size=20,     # defalt 16
	epochs=8, 	# defalt 5
	validation_data=(X_valid, y_valid))


Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 137s 7ms/step - loss: 0.1408 - acc: 0.9530 - val_loss: 0.0722 - val_acc: 0.9788
Epoch 2/8
20000/20000 [==============================] - 128s 6ms/step - loss: 0.0879 - acc: 0.9671 - val_loss: 0.0711 - val_acc: 0.9798
Epoch 3/8
20000/20000 [==============================] - 128s 6ms/step - loss: 0.0866 - acc: 0.9677 - val_loss: 0.0807 - val_acc: 0.9762
Epoch 4/8
20000/20000 [==============================] - 128s 6ms/step - loss: 0.0827 - acc: 0.9692 - val_loss: 0.0766 - val_acc: 0.9778
Epoch 5/8
20000/20000 [==============================] - 128s 6ms/step - loss: 0.0763 - acc: 0.9707 - val_loss: 0.0559 - val_acc: 0.9872
Epoch 6/8
20000/20000 [==============================] - 128s 6ms/step - loss: 0.0755 - acc: 0.9716 - val_loss: 0.0696 - val_acc: 0.9820
Epoch 7/8
20000/20000 [==============================] - 128s 6ms/step - loss: 0.0731 - acc: 0.9723 - val_loss: 0.0640 - val_acc:

In [7]:
for i in range(len(model.layers)):
    print(model.layers[i].name, i)

for layer in model.layers[770:]:	# total 783 layers
    layer.trainable = True


from keras import optimizers
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


input_1 0
lambda_1 1
conv2d_1 2
batch_normalization_1 3
activation_1 4
conv2d_2 5
batch_normalization_2 6
activation_2 7
conv2d_3 8
batch_normalization_3 9
activation_3 10
max_pooling2d_1 11
conv2d_4 12
batch_normalization_4 13
activation_4 14
conv2d_5 15
batch_normalization_5 16
activation_5 17
max_pooling2d_2 18
conv2d_9 19
batch_normalization_9 20
activation_9 21
conv2d_7 22
conv2d_10 23
batch_normalization_7 24
batch_normalization_10 25
activation_7 26
activation_10 27
average_pooling2d_1 28
conv2d_6 29
conv2d_8 30
conv2d_11 31
conv2d_12 32
batch_normalization_6 33
batch_normalization_8 34
batch_normalization_11 35
batch_normalization_12 36
activation_6 37
activation_8 38
activation_11 39
activation_12 40
mixed_5b 41
conv2d_16 42
batch_normalization_16 43
activation_16 44
conv2d_14 45
conv2d_17 46
batch_normalization_14 47
batch_normalization_17 48
activation_14 49
activation_17 50
conv2d_13 51
conv2d_15 52
conv2d_18 53
batch_normalization_13 54
batch_normalization_15 55
batch_norm

In [8]:
model.fit(X_train, y_train, batch_size=20, 	# first: 16
	epochs=8, 	# first: 5 
	validation_data=(X_valid, y_valid))

Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 138s 7ms/step - loss: 0.0772 - acc: 0.9719 - val_loss: 0.0385 - val_acc: 0.9912
Epoch 2/8
20000/20000 [==============================] - 131s 7ms/step - loss: 0.0679 - acc: 0.9743 - val_loss: 0.0382 - val_acc: 0.9912
Epoch 3/8
20000/20000 [==============================] - 131s 7ms/step - loss: 0.0637 - acc: 0.9761 - val_loss: 0.0382 - val_acc: 0.9914
Epoch 4/8
20000/20000 [==============================] - 131s 7ms/step - loss: 0.0688 - acc: 0.9749 - val_loss: 0.0379 - val_acc: 0.9914
Epoch 5/8
20000/20000 [==============================] - 131s 7ms/step - loss: 0.0616 - acc: 0.9759 - val_loss: 0.0376 - val_acc: 0.9914
Epoch 6/8
20000/20000 [==============================] - 131s 7ms/step - loss: 0.0671 - acc: 0.9743 - val_loss: 0.0375 - val_acc: 0.9908
Epoch 7/8
20000/20000 [==============================] - 131s 7ms/step - loss: 0.0601 - acc: 0.9761 - val_loss: 0.0362 - val_acc:

In [9]:
model.save('modelInceptionResNet_transfer_0_07.h5')

In [10]:
X_test = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
for i in tqdm(range(12500)):
    j = i+1
    X_test[i] = cv2.resize(cv2.imread('test/%d.jpg' % j), (img_height, img_width))

100%|██████████| 12500/12500 [01:48<00:00, 115.07it/s]


In [11]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

25000/25000 [==============================] - 143s 6ms/step


In [13]:
import pandas as pd
df = pd.read_csv("sample_submission.csv")

for i in range(12500):
	df.set_value(i, 'label', y_pred[i])

df.to_csv('pred_changePredictCsvMethond_180711.csv', index=None)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
